In [1]:
import os
import timeit
import numpy as np
import pandas as pd
import autokeras as ak
import tensorflow as tf
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [2]:
DATASET_LOCAL_PATH = "C:/Users/affiqazrin/Desktop/mmlspark/Data_FinalProject_READY4.csv"
df = pd.read_csv(DATASET_LOCAL_PATH)
    
ALL_COLS = ["age", #numerical
            "job", #categorical
            "marital", #categorical
            "education", #categorical
            "default", #categorical
            "housing", #categorical, binary
            "loan", #categorical, binary
            "contact", #categorical
            "day", #categorical
            "month", #categorical
            "duration", #numerical
            "campaign", #categorical
            "pdays", #numerical
            "previous", #numerical
            "poutcome", #categorical
            "deposit", #categorical, binary
           ]
    
NUMERICAL_COLS = ["age", #numerical
                  "duration", #numerical
                  "pdays", #numerical
                  "previous", #numerical
                 ]
    
CATEGORICAL_COLS = ["job", #categorical
                    "marital", #categorical
                    "education", #categorical
                    "default", #categorical
                    "housing", #categorical, binary
                    "loan", #categorical, binary
                    "contact", #categorical
                    "day", #categorical
                    "month", #categorical
                    "campaign", #categorical
                    "poutcome" #categorical
                   ]

TARGET_COL = ["deposit" #categorical, binary
             ]

le = LabelEncoder()
#TARGET_COL2 = le.fit_transform(df[TARGET_COL])
TARGET_COL2 = df[TARGET_COL].apply(LabelEncoder().fit_transform)

ohe = OneHotEncoder(handle_unknown='ignore')
CATEGORICAL_COLS2 = pd.DataFrame(ohe.fit_transform(df[CATEGORICAL_COLS]).toarray())

DATA_PROCESSED = pd.concat([df[NUMERICAL_COLS], CATEGORICAL_COLS2], axis=1)
mask = DATA_PROCESSED + TARGET_COL2
    
# Correlation_plot(model[mask], response_column)    
X = DATA_PROCESSED.values
y = TARGET_COL2.values.ravel()
    
sm = SMOTE(random_state=12)
X_resampled, y_resampled = sm.fit_sample(X, y)
    
X_train, X_test, y_train, y_test = train_test_split(X_resampled,
                                                    y_resampled,
                                                    test_size = 0.3,
                                                    random_state = 0)
print('Size of resampled data:')
print(' train shape... ', X_train.shape, y_train.shape)
print(' test shape.... ', X_test.shape, y_test.shape)

DATA_PROCESSED_HEADER=list(DATA_PROCESSED.columns.values)
n_features = len(DATA_PROCESSED_HEADER)
n_ALL_COLS = len(ALL_COLS)
n_NUMERICAL_COLS = len(NUMERICAL_COLS)
n_CATEGORICAL_COLS = len(CATEGORICAL_COLS)

Size of resampled data:
 train shape...  (15219, 94) (15219,)
 test shape....  (6523, 94) (6523,)


In [3]:
# Initialize the classifier.    
clf = ak.StructuredDataClassifier(max_trials=10,
                                  directory='tmp_dir',
                                  overwrite=True)

start_time = timeit.default_timer()
# x is the path to the csv file. y is the column name of the column to predict.
clf.fit(X_train, y_train)
stop_time = timeit.default_timer()

# Evaluate the accuracy of the found model.
accuracy = clf.evaluate(X_test, y_test)[1]
print('Accuracy: {accuracy}%'.format(accuracy=round(accuracy * 100, 2)))
print('Total time: {time} seconds.'.format(time=round(stop_time - start_time, 2)))

Trial 10 Complete [00h 00m 47s]
val_accuracy: 0.5011562705039978

Best val_accuracy So Far: 0.5011562705039978
Total elapsed time: 00h 07m 59s
INFO:tensorflow:Oracle triggered exit
476/476 [==============================] - 1s 3ms/step - loss: nan - accuracy: 0.5028
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: tmp_dir\structured_data_classifier\best_model\assets


204/204 [==============================] - 1s 3ms/step - loss: nan - accuracy: 0.4935
Accuracy: 49.35%
Total time: 538.74 seconds.


In [5]:
#Get the best performing model & Summarize the loaded model
model = clf.export_model()
model.summary()

#Save the best performing model to file
model.save('duplicate_autoKeras_SAVEDMODEL.tf')

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 94)]              0         
_________________________________________________________________
multi_category_encoding (Mul (None, 94)                0         
_________________________________________________________________
normalization (Normalization (None, 94)                189       
_________________________________________________________________
dense (Dense)                (None, 32)                3040      
_________________________________________________________________
re_lu (ReLU)                 (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                1056      
_________________________________________________________________
re_lu_1 (ReLU)               (None, 32)               

In [ ]:
from keras.models import load_model
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

model = load_model('duplicate_autoKeras_SAVEDMODEL.h5')
SVG(model_to_dot(model).create(prog='dot', format='svg'))